In [1]:
# special IPython command to prepare the notebook for matplotlib
%matplotlib inline 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely import geometry as geom
import plotly.plotly as py

#added this because it's indicated as useful but not imported
import urllib
from zipfile import ZipFile
from io import BytesIO
import os
import json
from bs4 import BeautifulSoup as bs
import re

from matplotlib import animation, rc
from IPython.display import display, HTML

In [2]:
with open("keys.json") as json_file:    
    keys = json.load(json_file)
census_key = keys["census"]["api_key"]

In [3]:
census_vars = json.loads(urllib.request.urlopen("http://api.census.gov/data/2014/acs5/variables.json").read().decode('utf-8'))["variables"]
income_brackets = dict([(key, value["label"]) for key, value in census_vars.items() if "B19001_" in key and key[-1] == "E"])

In [4]:
from us import states
from census import Census

def getincome20xx(income_brackets, years, census_key, zips):
    c = Census(census_key)
    df = pd.DataFrame()
    for year in years:
        tempget = c.acs5.zipcode((tuple([key[0] for key in income_brackets.items()])), Census.ALL, year=year)
        tempDF = pd.DataFrame(tempget)
        tempDF.columns = [income_brackets[tempDF.columns[i]] if i < len(income_brackets) else tempDF.columns[i] for i in range(len(tempDF.columns))]
        tempDF = tempDF[tempDF[tempDF.columns[-1]].isin(zips)]
        tempDF["year"] = year
        df = df.append(tempDF)
    return df

I forgot I wrote this. dope. :D

In [5]:
zipgeojson = "https://github.com/YuHuaCheng/Map/raw/master/data/nyc-zip-code-tabulation-areas-polygons.geojson"

# save the NYC map data under "data" folder
zipsfile = os.path.join("data",zipgeojson.split("/")[-1])

# read the saved NYC map data file
try:
    NY = gpd.read_file(zipsfile) # NYC map data with Zipcode
except OSError as e:
    urllib.request.urlretrieve(zipgeojson, zipsfile)
    
    #zips geojson had a var assignment that caused problems
    fin      = open(zipsfile)
    
    lines    = [line for line in fin]
    lines[0] = lines[0].replace("var nycdata = ", "")
    
    fin.close()
    
    fout = open(zipsfile, "w")
    for line in lines:
        fout.write(line)
    fout.flush()
    fout.close()
    
    NY = gpd.read_file(zipsfile)
    
NY.shape # NYC is composed of 262 subarea on the map data

(262, 12)

In [6]:
QN = NY[NY.borough == "Brooklyn"]
zips_QN = QN.postalCode.unique()

In [7]:
theDF = getincome20xx(income_brackets,range(2011,2015), census_key, zips_QN)

In [8]:
newcols = ["total", "0", "10", "15", "20", "25", "30", "35", "40", "45", "50", "60", "75", "100", "125", "150", "200", "ZIP", "year"]
theDF.columns = newcols
theDF[newcols] = theDF[newcols].apply(pd.to_numeric)
theDF.head()

,total,0,10,15,20,25,30,35,40,45,50,60,75,100,125,150,200,ZIP,year
2694,23254,1656,706,618,710,403,643,596,686,547,1303,1853,2929,1954,1456,2219,4975,11201,2011
2695,27145,2740,1244,1525,1486,1352,1158,1397,1453,1449,1876,2761,3418,2316,1410,994,566,11203,2011
2696,24979,1939,1920,1705,1564,1580,1373,1333,1223,1038,2025,2536,2655,1572,900,1043,573,11204,2011
2697,14033,2131,1017,874,866,659,614,488,570,324,889,1193,1255,1160,581,640,772,11205,2011
2698,27668,5551,2800,2612,1950,1551,1643,1079,1405,1172,1669,1675,1911,1170,753,478,249,11206,2011


In [9]:
theDF.to_csv(os.path.join('data', "BK_incomeDF.csv"))

In [10]:
QN_files = ["https://www.huduser.gov/portal/datasets/fmr/fmr2010f/FY2010_4050_Final_PostRDDs.xls",
         "https://www.huduser.gov/portal/datasets/fmr/fmr2011f/fy2011_equiv_zc_rents_acs.xls",
         "https://www.huduser.gov/portal/datasets/fmr/fmr2012f/small_area_fmrs_fy2012.xls",
         "https://www.huduser.gov/portal/datasets/fmr/fmr2013p/small_area_fmrs_fy2013.xls",
         "https://www.huduser.gov/portal/datasets/fmr/fmr2014f/small_area_fmrs_fy2014.xls"]

In [11]:
from JP_Cache import get_cached_or_dl
from IPython.display import display

In [12]:
rent_per_year = get_cached_or_dl(QN_files)

In [13]:
for i in range(len(rent_per_year)):
    for j in rent_per_year[i].columns.values:
        if j == 'zip' or j == 'ZIP':
            rent_per_year[i].rename(columns={j: 'ZIP' }, inplace=True)
        if j == 'countyname' or j == 'County Name' or j == 'County_Name':
            rent_per_year[i].rename(columns={j: 'CountyName' }, inplace=True)
        if j == 'State Name' or j == 'state_alpha':
            rent_per_year[i].rename(columns={j: 'StateName' }, inplace=True)
        if j == 'area_rent_br0' or j == 'fmr0':
            rent_per_year[i].rename(columns={j: 'rent0' }, inplace=True)
        if j == 'area_rent_br1' or j == 'fmr1':
            rent_per_year[i].rename(columns={j: 'rent1' }, inplace=True)
        if j == 'area_rent_br2' or j == 'fmr2':
            rent_per_year[i].rename(columns={j: 'rent2' }, inplace=True)
        if j == 'area_rent_br3' or j == 'fmr3':
            rent_per_year[i].rename(columns={j: 'rent3' }, inplace=True)
        if j == 'area_rent_br4' or j == 'fmr4':
            rent_per_year[i].rename(columns={j: 'rent4' }, inplace=True)
        if j == 'state':
            rent_per_year[i].rename(columns={j: 'State' }, inplace=True)

In [14]:
#df_14 = 
#df_14 = df_14[['StateName','CountyName','rent0','rent1','rent2','rent3','rent4','ZIP']]
#df_14.head()

In [15]:
rent_per_year[1]["year"] = 2011
rent_per_year[2]["year"] = 2012
rent_per_year[3]["year"] = 2013
rent_per_year[4]["year"] = 2014
rent_per_year = rent_per_year[1:] # drop 2010

In [16]:
rent_per_year_QN = []
for rentdf in rent_per_year:
    #display(rentdf.head())
    rent_per_year_QN.append(rentdf[(rentdf.CountyName == 'Kings County') & (rentdf.State == 36)])

In [17]:
rent_per_year_QN[0]["StateName"] = "New York"

/home/stuff/anaconda3/envs/DataScience-Project/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [18]:
for rentdf in rent_per_year_QN:
    display(rentdf.head())

,cbsamet,State,county,ZIP,CountyName,CBSA_Name,rent0,rent1,rent2,rent3,rent4,year,StateName
12321,35620,36,47,10016,Kings County,"New York-Northern New Jersey-Long Island, NY-N...",1060,1130,1280,1590,1760,2011,New York
12322,35620,36,47,11201,Kings County,"New York-Northern New Jersey-Long Island, NY-N...",1530,1630,1840,2280,2520,2011,New York
12323,35620,36,47,11203,Kings County,"New York-Northern New Jersey-Long Island, NY-N...",1040,1110,1250,1550,1720,2011,New York
12324,35620,36,47,11204,Kings County,"New York-Northern New Jersey-Long Island, NY-N...",1070,1140,1290,1600,1770,2011,New York
12325,35620,36,47,11205,Kings County,"New York-Northern New Jersey-Long Island, NY-N...",1170,1250,1410,1750,1930,2011,New York


,State,StateName,County,CountyName,CBSA,CBSA Name,ZIP,rent0,rent1,rent2,rent3,rent4,year
13751,36,New York,47,Kings County,35620,"New York-Northern New Jersey-Long Island, NY-N...",11201,1500,1640,1870,2360,2860,2012
13752,36,New York,47,Kings County,35620,"New York-Northern New Jersey-Long Island, NY-N...",11203,980,1070,1220,1540,1860,2012
13753,36,New York,47,Kings County,35620,"New York-Northern New Jersey-Long Island, NY-N...",11204,1050,1150,1310,1650,2000,2012
13754,36,New York,47,Kings County,35620,"New York-Northern New Jersey-Long Island, NY-N...",11205,1150,1260,1440,1820,2200,2012
13755,36,New York,47,Kings County,35620,"New York-Northern New Jersey-Long Island, NY-N...",11206,890,970,1110,1400,1690,2012


,State,StateName,County,CountyName,CBSA,CBSA Name,ZIP,rent0,rent1,rent2,rent3,rent4,year
14293,36,New York,47,Kings County,35620,"New York-Northern New Jersey-Long Island, NY-N...",11201,1550,1620,1920,2500,2940,2013
14294,36,New York,47,Kings County,35620,"New York-Northern New Jersey-Long Island, NY-N...",11202,1070,1120,1330,1730,2040,2013
14295,36,New York,47,Kings County,35620,"New York-Northern New Jersey-Long Island, NY-N...",11203,1010,1050,1250,1630,1920,2013
14296,36,New York,47,Kings County,35620,"New York-Northern New Jersey-Long Island, NY-N...",11204,1070,1120,1330,1730,2040,2013
14297,36,New York,47,Kings County,35620,"New York-Northern New Jersey-Long Island, NY-N...",11205,1270,1320,1570,2040,2410,2013


,State,StateName,County,CountyName,CBSA,CBSA Name,ZIP,rent0,rent1,rent2,rent3,rent4,year
14328,36,New York,47,Kings County,35620,"New York-Northern New Jersey-Long Island, NY-N...",11201,1540,1610,1910,2480,2930,2014
14329,36,New York,47,Kings County,35620,"New York-Northern New Jersey-Long Island, NY-N...",11202,1070,1110,1320,1720,2020,2014
14330,36,New York,47,Kings County,35620,"New York-Northern New Jersey-Long Island, NY-N...",11203,1000,1050,1240,1610,1900,2014
14331,36,New York,47,Kings County,35620,"New York-Northern New Jersey-Long Island, NY-N...",11204,1070,1110,1320,1720,2020,2014
14332,36,New York,47,Kings County,35620,"New York-Northern New Jersey-Long Island, NY-N...",11205,1260,1320,1560,2030,2390,2014


In [19]:
rentQN = pd.DataFrame()
for rentdf in rent_per_year_QN:
    tempdf = rentdf[['State','CountyName','rent0','rent1','rent2','rent3','rent4','ZIP', 'year']]
    print((len(rentdf), len(tempdf)))
    rentQN = rentQN.append(tempdf, ignore_index=True)
print(len(rentQN))
rentQN.head()

(39, 39)
(45, 45)
(41, 41)
(41, 41)
166


,State,CountyName,rent0,rent1,rent2,rent3,rent4,ZIP,year
0,36,Kings County,1060,1130,1280,1590,1760,10016,2011
1,36,Kings County,1530,1630,1840,2280,2520,11201,2011
2,36,Kings County,1040,1110,1250,1550,1720,11203,2011
3,36,Kings County,1070,1140,1290,1600,1770,11204,2011
4,36,Kings County,1170,1250,1410,1750,1930,11205,2011


In [20]:
rentQN[['rent0','rent1','rent2','rent3','rent4',"ZIP", "year"]] = rentQN[['rent0','rent1','rent2','rent3','rent4',"ZIP", "year"]].apply(pd.to_numeric)
rentQN.head()

,State,CountyName,rent0,rent1,rent2,rent3,rent4,ZIP,year
0,36,Kings County,1060,1130,1280,1590,1760,10016,2011
1,36,Kings County,1530,1630,1840,2280,2520,11201,2011
2,36,Kings County,1040,1110,1250,1550,1720,11203,2011
3,36,Kings County,1070,1140,1290,1600,1770,11204,2011
4,36,Kings County,1170,1250,1410,1750,1930,11205,2011


In [21]:
rentQN.to_csv(os.path.join('data', "BK_rentDF.csv"))

In [22]:
merged = pd.merge(theDF,rentQN,how = "inner", on =['year', 'ZIP'])
merged.head()

,total,0,10,15,20,25,30,35,40,45,...,200,ZIP,year,State,CountyName,rent0,rent1,rent2,rent3,rent4
0,23254,1656,706,618,710,403,643,596,686,547,...,4975,11201,2011,36,Kings County,1530,1630,1840,2280,2520
1,27145,2740,1244,1525,1486,1352,1158,1397,1453,1449,...,566,11203,2011,36,Kings County,1040,1110,1250,1550,1720
2,24979,1939,1920,1705,1564,1580,1373,1333,1223,1038,...,573,11204,2011,36,Kings County,1070,1140,1290,1600,1770
3,14033,2131,1017,874,866,659,614,488,570,324,...,772,11205,2011,36,Kings County,1170,1250,1410,1750,1930
4,27668,5551,2800,2612,1950,1551,1643,1079,1405,1172,...,249,11206,2011,36,Kings County,1040,1110,1250,1550,1720


In [23]:
merged.to_csv("data/BK_merged.csv")